In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings 
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *
from statsforecast import StatsForecast
from statsforecast.models import (
    Naive,WindowAverage, ARIMA, 
    AutoARIMA,SeasonalNaive,HoltWinters,
    CrostonClassic as Croston, HistoricAverage,DynamicOptimizedTheta as DOT,
    SeasonalNaive
)
from dataclasses import dataclass
from typing import Optional, List, Dict, Tuple, Union
from itertools import product


warnings.filterwarnings("ignore")  # To ignore warnings from pandas/numpy

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Color palette for up to 4 models
MODEL_COLORS = {
    0: '#E94F37',  # Red
    1: '#2E86AB',  # Blue
    2: '#28A745',  # Green
    3: '#9B59B6',  # Purple
}
HISTORICAL_COLOR = "#898992" 

In [ ]:
user="Lilian"

### Dataclass and Config

In [11]:
@dataclass
class ForecastConfig:
    
    # Forecast parameters
    h: int = 8                          
    season_length: int = 4              
    
    # Cross-validation parameters
    n_windows: int = 2                  
    step_size: Optional[int] = None     
    
    # Train-test split parameters
    train_size: Optional[int] = None    # Use all available data except test
    test_size: Optional[int] = None     # Auto-set to h in __post_init__
    
    # Plotting parameters
    n_samples: int = 4                  # Plot 4 random samples
    models_to_plot: Optional[List[str]] = None  # 
    
    # Other settings
    confidence_level: int = 95          # 95% confidence intervals
    n_jobs: int = -1                    
    
    def __post_init__(self):
        
        if self.step_size is None:
            self.step_size = self.h
        if self.test_size is None:
            self.test_size = self.h
        if self.models_to_plot is None:
            self.models_to_plot = ['Naive', 'ARIMA_manual', 'SARIMA']

### SDUD with pop - Statistical Models

In [54]:
#Defining functions used above but this time for exogenous feature (pop)
def load_data_exog(filepath, states=None):

    df = pd.read_csv(filepath)
    df['ds'] = pd.to_datetime(df['Period'], errors='coerce')
    
    df = df[(df['ds'] >= '2017-01-01') & (df['ds'] <= '2024-10-01')].copy()
    
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df = df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].unique()} state(s)")
        if len(df) == 0:
            raise ValueError("No data available after filtering by states.")
    
    df['unique_id'] = df['State'] + '_' + df['ATC2 Class']
    df = df.sort_values(['unique_id','ds']).reset_index(drop=True)

    df_units = df[['unique_id', 'ds', 'Units Reimbursed']].copy()
    df_units.columns = ['unique_id', 'ds', 'y']
    df_units = df_units.dropna(subset=['y'])  

    df_prescriptions = df[['unique_id', 'ds', 'Number of Prescriptions']].copy()
    df_prescriptions.columns = ['unique_id', 'ds', 'y']
    df_prescriptions = df_prescriptions.dropna(subset=['y'])  # ← Only drop NaN in y

    return df_units, df_prescriptions, df

def pop_scenarios_exog(filepath,states=None):

    df = pd.read_csv(filepath)
    df['ds'] = pd.to_datetime(df['Period'], errors='coerce')
    
    if states is not None:
        if isinstance(states, str):
            states = [states]
        df = df[df['State'].isin(states)].copy()
    
    df['unique_id'] = df['State'] + '_' + df['ATC2 Class']
    df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)
    
    df_historical = df[df['ds'] <= '2024-10-01'].copy()
    df_future = df[df['ds'] > '2024-10-01'].copy()

    pop_historical = df_historical[['unique_id', 'ds', 'population']].copy()
    pop_historical.columns = ['unique_id', 'ds', 'population']

    scenarios={}
    pop_future_point = df_future[['unique_id', 'ds', 'population']].copy()
    pop_future_point.columns = ['unique_id', 'ds', 'population']
    scenarios['point'] = pd.concat([pop_historical, pop_future_point], ignore_index=True)
    
    # Scenario 2: Lower bound (uses Forecast_low_95 column)
    pop_future_low = df_future[['unique_id', 'ds', 'Forecast_low_95']].copy()
    pop_future_low.columns = ['unique_id', 'ds', 'population']
    scenarios['low_95'] = pd.concat([pop_historical, pop_future_low], ignore_index=True)
    
    # Scenario 3: Upper bound (uses Forecast_high_95 column)
    pop_future_high = df_future[['unique_id', 'ds', 'Forecast_high_95']].copy()
    pop_future_high.columns = ['unique_id', 'ds', 'population']
    scenarios['high_95'] = pd.concat([pop_historical, pop_future_high], ignore_index=True)
    
    # Verify we have future data
    for scenario_name, scenario_df in scenarios.items():
        future_count = len(scenario_df[scenario_df['ds'] > '2024-10-01'])
        print(f"   {scenario_name}: {future_count} future population records")
    
    return scenarios

def get_models_exog(config, use_exog=True):
    
    if use_exog:
        models = [
            # Baseline models (ignore exog)
            Naive(),
            SeasonalNaive(season_length=config.season_length),
            HistoricAverage(),
            WindowAverage(window_size=4),
            
            # Models with exogenous support
            AutoARIMA(
                seasonal=True, 
                season_length=config.season_length, 
                alias="SARIMAX"
            ),
            AutoARIMA(
                seasonal=False,
                season_length=config.season_length,
                alias="ARIMAX"
            ),
            # Note: Can add more exog-aware models here
        ]
    else:
        # Standard models without exog
        models = [
            Naive(),
            HistoricAverage(),
            WindowAverage(window_size=4),
            SeasonalNaive(season_length=config.season_length),
            ARIMA(order=(1, 1, 1), alias="ARIMA_manual"),
            AutoARIMA(seasonal=True, season_length=config.season_length, alias="SARIMA"),
        ]
    
    return models

def evaluate_train_test_exog(df,target_name,config,population_df=None):
    
    # Split train/test
    if config.train_size is None:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[:-config.test_size]
        ).reset_index(drop=True)
    else:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[-(config.train_size + config.test_size):-config.test_size]
        ).reset_index(drop=True)
    
    test = df.groupby('unique_id').apply(
        lambda x: x.iloc[-config.test_size:]
    ).reset_index(drop=True)
    
    # Get models
    use_exog = population_df is not None
    models = get_models_exog(config, use_exog=use_exog)
    
    # Initialize StatsForecast
    sf = StatsForecast(
        models=models,
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    # Fit and predict with/without exogenous variables
    if use_exog:

        if 'population' in train.columns:
            train = train.drop(columns=['population'])
        if 'population' in test.columns:
            test = test.drop(columns=['population'])

        # Prepare exogenous data for train and test
        pop_train = population_df[population_df['ds'].isin(train['ds'])].copy()
        pop_test = population_df[population_df['ds'].isin(test['ds'])].copy()
        
        # Merge population with train data
        train_with_pop = train.merge(pop_train, on=['unique_id', 'ds'], how='left')
        
        # Fit with exogenous
        sf.fit(df=train_with_pop[['unique_id', 'ds', 'y', 'population']])
        
        # Predict with future exogenous
        test_with_pop = test.merge(pop_test, on=['unique_id', 'ds'], how='left')
        preds = sf.predict(h=config.h, X_df=test_with_pop[['unique_id', 'ds', 'population']])
    else:
        sf.fit(df=train)
        preds = sf.predict(h=config.h)
    
    # Merge predictions with actuals
    preds_df = pd.merge(test, preds.reset_index(), on=['ds', 'unique_id'], how='left')
    
    # Evaluate
    model_cols = [col for col in preds.columns if col not in ['unique_id', 'ds']]
    eval_df = evaluate(preds_df, metrics=[mae, mse, smape], models=model_cols)
    
    # Add best model
    mae_df = eval_df[eval_df['metric'] == 'mae'].copy()
    mae_df['best_model'] = mae_df[model_cols].idxmin(axis=1)
    
    exog_label = " (with population)" if use_exog else " (no exog)"
    print(f"\n📈 Best Models (Train-Test Split - {target_name}{exog_label}):")
    print(mae_df['best_model'].value_counts())
    
    return eval_df, preds_df, train, test

def CV_with_exog(df,target_name,config,population_df=None):
    
    use_exog = population_df is not None
    models = get_models_exog(config, use_exog=use_exog)
    
    sf = StatsForecast(
        models=models,
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    print(f"   Running cross-validation...")
    
    # Cross-validation with/without exogenous
    if use_exog:

        df_for_cv = df.copy()
        if 'population' in df_for_cv.columns:
            df_for_cv = df_for_cv.drop(columns=['population'])
            
        df_with_pop = df.merge(population_df, on=['unique_id', 'ds'], how='left')
        cv_df = sf.cross_validation(
            df=df_with_pop[['unique_id', 'ds', 'y', 'population']],
            h=config.h,
            n_windows=config.n_windows,
            step_size=config.step_size
        )
    else:
        cv_df = sf.cross_validation(
            df=df,
            h=config.h,
            n_windows=config.n_windows,
            step_size=config.step_size
        )
    
    # Evaluate per cutoff
    exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'population']
    model_cols = [col for col in cv_df.columns if col not in exclude_cols]
    
    all_results = []
    
    for cutoff in cv_df['cutoff'].unique():
        cutoff_data = cv_df[cv_df['cutoff'] == cutoff]
        
        cutoff_eval = evaluate(cutoff_data, metrics=[mae, mse, smape], models=model_cols)
        cutoff_eval['cutoff'] = cutoff
        cutoff_eval['best_model'] = cutoff_eval[model_cols].idxmin(axis=1)
        cutoff_eval['best_value'] = cutoff_eval[model_cols].min(axis=1)
        
        all_results.append(cutoff_eval)
        
        cutoff_mae = cutoff_eval[cutoff_eval['metric'] == 'mae']
        print(f"\n   Cutoff {cutoff.strftime('%Y-%m-%d')} (MAE best models):")
        print(f"   {cutoff_mae['best_model'].value_counts().to_dict()}")
    
    eval_df = pd.concat(all_results, ignore_index=True)
    
    mae_overall = eval_df[eval_df['metric'] == 'mae']
    exog_label = " (with population)" if use_exog else ""
    print(f"\n📈 Overall Best Models{exog_label}:")
    print(mae_overall['best_model'].value_counts())
    
    return eval_df, cv_df

def generate_future_forecasts(df_historical, target_name, config, population_scenario, horizon=None,save_path=None):
    
    if horizon is None:
        horizon = config.h
    use_exog = population_scenario is not None
    models = get_models_exog(config, use_exog=use_exog)
    
    sf = StatsForecast(
        models=models,
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    # Train on full historical data
    if use_exog:
        # Get historical population
        pop_historical = population_scenario[
            population_scenario['ds'] <= df_historical['ds'].max()
        ].copy()
        
        df_train = df_historical.merge(pop_historical, on=['unique_id', 'ds'], how='left')
        sf.fit(df=df_train[['unique_id', 'ds', 'y', 'population']])
        
        # Get future population for forecasting
        last_date = df_historical['ds'].max()
        future_dates = pd.date_range(
            start=last_date + pd.DateOffset(months=3),
            periods=horizon,
            freq='QS'
        )
        
        # Create future exogenous dataframe
        future_exog_list = []
        for uid in df_historical['unique_id'].unique():
            for date in future_dates:
                pop_val = population_scenario[
                    (population_scenario['unique_id'] == uid) & 
                    (population_scenario['ds'] == date)
                ]['population'].values
                
                if len(pop_val) > 0:
                    future_exog_list.append({
                        'unique_id': uid,
                        'ds': date,
                        'population': pop_val[0]
                    })
        
        future_exog_df = pd.DataFrame(future_exog_list)
        forecasts_df = sf.predict(h=horizon, X_df=future_exog_df)
    else:
        sf.fit(df=df_historical,)
        forecasts_df = sf.predict(h=horizon)
    
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        filename = os.path.join(
            save_path, 
            f'{target_name.lower().replace(" ", "_")}_future_h{horizon}.csv'
        )
        forecasts_df.reset_index().to_csv(filename, index=False)
        print(f"✅ Saved: {filename}")
    
    return forecasts_df

def forecast_with_population_scenarios(filepath, target_col, states=None, config=None, horizons=None, save_path=None):
   
    if config is None: config = ForecastConfig()

    if horizons is None: horizons = [config.h]

    print(f"\nFORECASTING: {target_col}, Horizons: {horizons}")

    df_units, df_prescriptions, df_full = load_data_exog(filepath, states)

    df = df_units if target_col == 'Units Reimbursed' else df_prescriptions

    print(f"   Unique series: {df['unique_id'].nunique()}, Range: {df['ds'].min()} to {df['ds'].max()}")
    
    pop_scenarios = pop_scenarios_exog(filepath, states)
    results = {'point': {}, 'low_95': {}, 'high_95': {}}
    
    for scenario_name, pop_df in pop_scenarios.items():
        print(f"\n--- SCENARIO: {scenario_name.upper()} ---")
        for h in horizons:
            forecasts = generate_future_forecasts(df, target_col, config, pop_df, horizon=h,
                                                  save_path=os.path.join(save_path, scenario_name) if save_path else None)
            results[scenario_name][f'h{h}'] = {
                'forecasts': forecasts,
                'df_historical': df.copy()
            }
    print("\n✅ COMPLETE!")
    return results
    print("="*70)
    
    return results                

In [55]:
def sensitivity_analysis_with_exog(filepath, target_col, states=None, param_grid=None, production_horizons=[4, 8],
    population_scenario='point',  # 'point', 'low_95', or 'high_95'
    save_path=None):

    # Default parameter grid
    if param_grid is None:
        param_grid = {
            'h': [4, 8],
            'test_size': [4, 8],
            'train_size': [None, 20, 28],
            'n_windows': [2, 3, 4],
            'step_size': [4],
        }
    
    print("=" * 80); print(f"SENSITIVITY ANALYSIS WITH EXOGENOUS VARIABLES: {target_col}")
    print(f"Population Scenario: {population_scenario.upper()}"); print(f"Testing {len(list(product(*param_grid.values())))} configurations")
    print("=" * 80); print("\nPhase 1: Loading and preparing data...")
    
    df_units, df_prescriptions, df_full = load_data_exog(filepath, states)
    df = df_units if target_col == 'Units Reimbursed' else df_prescriptions
    target_name = target_col
    
    print(f"\nUnique series: {df['unique_id'].nunique()}")
    print(f"\nDate range: {df['ds'].min()} to {df['ds'].max()}")
    print(f"\nTotal observations: {len(df)}")
    
    # Get population scenarios
    pop_scenarios = pop_scenarios_exog(filepath, states)
    pop_df = pop_scenarios[population_scenario]
    
    print(f"\nPopulation scenario loaded: {population_scenario}")
    print(f"\nPopulation records: {len(pop_df)}")
    
    print("\nPhase 2: Running sensitivity analysis...")
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))
    
    all_results = []
    all_errors = []  # Simplified error tracking - mean per config
    
    for i, combo in enumerate(combinations):
        params = dict(zip(param_names, combo))
        
        # Skip invalid combinations
        if params.get('test_size') and params.get('h'):
            if params['test_size'] < params['h']:
                continue
        
        if params['test_size'] != params['h']:
            continue

        print(f"\n{'─' * 80}"); print(f"Configuration {i+1}/{len(combinations)}: {params}"); print('─' * 80)
        
        try:
            config = ForecastConfig(
                h=params.get('h', 8),
                train_size=params.get('train_size'),
                test_size=params.get('test_size'),
                n_windows=params.get('n_windows', 2),
                step_size=params.get('step_size'),
                season_length=4,
                n_jobs=-1
            )
            
            # Train-Test Evaluation
            print("\nRunning train-test evaluation...")
            eval_tt, preds_tt, train_df, test_df = evaluate_train_test_exog(
                df, target_name, config, pop_df
            )
            
            # Cross-Validation
            print("\nRunning Cross-Validation...")
            eval_cv, cv_df = CV_with_exog(
                df, target_name, config, pop_df
            )
            
            exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'best_model', 'best_value', 'population']
            model_cols = [col for col in eval_cv.columns if col not in exclude_cols]
            
            # Extract MAE results
            mae_cv = eval_cv[eval_cv['metric'] == 'mae'].copy()
            mae_traintest = eval_tt[eval_tt['metric'] == 'mae'].copy()
            
            # Add best_model if not present
            if 'best_model' not in mae_traintest.columns:
                mae_traintest['best_model'] = mae_traintest[model_cols].idxmin(axis=1)
            
            # Collect Results Per Unique_ID 
            for uid in df['unique_id'].unique():
                uid_cv = mae_cv[mae_cv['unique_id'] == uid]
                uid_traintest = mae_traintest[mae_traintest['unique_id'] == uid]
                
                # CV analysis
                cv_best_model = uid_cv['best_model'].mode().iloc[0] if len(uid_cv) > 0 else None
                cv_best_count = (uid_cv['best_model'] == cv_best_model).sum()
                cv_total = len(uid_cv)
                cv_consistency = cv_best_count / cv_total if cv_total > 0 else 0
                
                # Train-test analysis
                traintest_best_model = uid_traintest['best_model'].iloc[0] if len(uid_traintest) > 0 else None
                
                # Build result row
                result_row = {
                    'unique_id': uid,
                    'config_id': i + 1,
                    'population_scenario': population_scenario,
                    **params,
                    'cv_best_model': cv_best_model,
                    'cv_consistency': cv_consistency,
                    'traintest_best_model': traintest_best_model,
                    'cv_traintest_agree': cv_best_model == traintest_best_model,
                }
                
                # Add MAE values for each model
                for model in model_cols:
                    cv_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    tt_mae = uid_traintest[model].iloc[0] if len(uid_traintest) > 0 and model in uid_traintest.columns else None
                    
                    result_row[f'{model}_cv_mae'] = cv_mae
                    result_row[f'{model}_tt_mae'] = tt_mae
                
                all_results.append(result_row)
                
                # One row per unique_id per configuration with MEAN CV MAE and Train-Test MAE
                error_row = {
                    'config_id': i + 1,
                    'unique_id': uid,
                    'population_scenario': population_scenario,
                    **params,
                    'n_cv_windows': cv_total,
                }
                
                # Add mean CV MAE and Train-Test MAE for each model
                for model in model_cols:
                    # Mean MAE across all CV windows
                    cv_mean_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    error_row[f'{model}_cv_mean_mae'] = cv_mean_mae
                    
                    # Train-Test MAE
                    tt_mae = uid_traintest[model].iloc[0] if len(uid_traintest) > 0 and model in uid_traintest.columns else None
                    error_row[f'{model}_tt_mae'] = tt_mae
                
                all_errors.append(error_row)
            
            print(f"   ✅ Configuration {i+1} completed successfully")
            
        except Exception as e:
            print(f"   ❌ Error in configuration {i+1}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    # Convert to DataFrames
    results_df = pd.DataFrame(all_results)
    errors_df = pd.DataFrame(all_errors)
    
    print(f"\n✅ Sensitivity analysis complete!")
    print(f"\nTotal configurations tested: {results_df['config_id'].nunique()}")
    print(f"\nTotal unique_ids analyzed: {results_df['unique_id'].nunique()}")
    
    print("\nPhase 3: Identifying best models per horizon...")
    
    recommendations_per_horizon = {}
    
    for horizon in production_horizons:
        print(f"\nAnalyzing horizon h={horizon}...")
        
        horizon_results = results_df[results_df['h'] == horizon].copy()
        
        if len(horizon_results) == 0:
            print(f"   ⚠️ No results for h={horizon}")
            continue
        
        horizon_recommendations = []
        
        for uid in horizon_results['unique_id'].unique():
            uid_data = horizon_results[horizon_results['unique_id'] == uid]
            
            # Most frequent CV best model
            cv_mode = uid_data['cv_best_model'].mode()
            cv_best = cv_mode.iloc[0] if len(cv_mode) > 0 else None
            cv_freq = (uid_data['cv_best_model'] == cv_best).sum() / len(uid_data)
            
            # Most frequent train-test best model
            tt_mode = uid_data['traintest_best_model'].mode()
            tt_best = tt_mode.iloc[0] if len(tt_mode) > 0 else None
            tt_freq = (uid_data['traintest_best_model'] == tt_best).sum() / len(uid_data)
            
            # Average consistency
            avg_consistency = uid_data['cv_consistency'].mean()
            
            # Determine recommendation
            if cv_best == tt_best and cv_freq >= 0.7 and avg_consistency >= 0.7:
                confidence = 'High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree with high frequency and consistency"
            elif cv_best == tt_best and cv_freq >= 0.5:
                confidence = 'Medium-High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree with moderate frequency"
            elif cv_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = cv_best
                reason = f"CV favors {cv_best} ({cv_freq:.0%})"
            elif tt_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = tt_best
                reason = f"Train-Test favors {tt_best} ({tt_freq:.0%})"
            else:
                confidence = 'Low'
                # Use lowest average CV MAE
                mae_cols = [col for col in uid_data.columns if col.endswith('_cv_mae')]
                if mae_cols:
                    avg_maes = uid_data[mae_cols].mean()
                    recommended_model = avg_maes.idxmin().replace('_cv_mae', '')
                else:
                    recommended_model = cv_best
                reason = "No clear winner - using lowest average CV MAE"
            
            # Get average MAE
            rec_mae_col = f'{recommended_model}_cv_mae'
            avg_mae = uid_data[rec_mae_col].mean() if rec_mae_col in uid_data.columns else None
            
            recommendation = {
                'unique_id': uid,
                'horizon': horizon,
                'recommended_model': recommended_model,
                'confidence': confidence,
                'reason': reason,
                'cv_best_model': cv_best,
                'cv_frequency': cv_freq,
                'cv_consistency': avg_consistency,
                'traintest_best_model': tt_best,
                'avg_cv_mae': avg_mae,
                'population_scenario': population_scenario
            }
            
            horizon_recommendations.append(recommendation)
            
            mae_display = f"{avg_mae:,.0f}" if pd.notna(avg_mae) else "N/A"
            print(f"   {uid}: {recommended_model} (confidence: {confidence}, MAE: {mae_display})")
        
        recommendations_per_horizon[horizon] = pd.DataFrame(horizon_recommendations)
    
    print("\nPhase 4: Generating production forecasts...")
    
    production_forecasts = {}
    
    for horizon in production_horizons:
        if horizon not in recommendations_per_horizon:
            continue
        
        print(f"\nGenerating forecasts for h={horizon}...")
        
        recommendations = recommendations_per_horizon[horizon]
        
        # Configuration for production
        config_prod = ForecastConfig(
            h=horizon,
            season_length=4,
            confidence_level=95,
            n_jobs=-1
        )
        
        # Train all models on full historical data
        models = get_models_exog(config_prod, use_exog=True)
        
        sf = StatsForecast(
            models=models,
            freq='QS',
            n_jobs=-1,
            fallback_model=SeasonalNaive(season_length=4)
        )
        
        # Prepare data with population
        pop_historical = pop_df[pop_df['ds'] <= df['ds'].max()].copy()
        df_train = df.merge(pop_historical, on=['unique_id', 'ds'], how='left')
        
        # Fit on historical data
        sf.fit(df=df_train[['unique_id', 'ds', 'y', 'population']])
        
        # Prepare future population
        last_date = df['ds'].max()
        future_dates = pd.date_range(
            start=last_date + pd.DateOffset(months=3),
            periods=horizon,
            freq='QS'
        )
        
        future_exog_list = []
        for uid in df['unique_id'].unique():
            for date in future_dates:
                pop_val = pop_df[
                    (pop_df['unique_id'] == uid) & 
                    (pop_df['ds'] == date)
                ]['population'].values
                
                if len(pop_val) > 0:
                    future_exog_list.append({
                        'unique_id': uid,
                        'ds': date,
                        'population': pop_val[0]
                    })
        
        future_exog_df = pd.DataFrame(future_exog_list)
        
        # Generate forecasts
        forecasts_df = sf.predict(h=horizon, X_df=future_exog_df)
        
        # Create best model forecasts
        best_forecasts = []
        
        for uid in df['unique_id'].unique():
            uid_forecasts = forecasts_df.reset_index()
            uid_forecasts = uid_forecasts[uid_forecasts['unique_id'] == uid]
            
            uid_rec = recommendations[recommendations['unique_id'] == uid]
            if len(uid_rec) == 0:
                print(f"   ⚠️ No recommendation for {uid}, skipping")
                continue
            
            best_model = uid_rec['recommended_model'].iloc[0]
            
            for _, row in uid_forecasts.iterrows():
                best_row = {
                    'unique_id': uid,
                    'ds': row['ds'],
                    'recommended_model': best_model,
                    'forecast': row.get(best_model, np.nan),
                    'forecast_lo_95': row.get(f'{best_model}-lo-95', np.nan),
                    'forecast_hi_95': row.get(f'{best_model}-hi-95', np.nan),
                    'population_scenario': population_scenario
                }
                best_forecasts.append(best_row)
        
        best_forecasts_df = pd.DataFrame(best_forecasts)
        
        production_forecasts[horizon] = {
            'all_models': forecasts_df,
            'best_model': best_forecasts_df,
            'recommendations': recommendations
        }
        
        print(f"   ✅ Generated {len(best_forecasts_df)} forecast periods")
        print(f"   ✅ Models used: {recommendations['recommended_model'].value_counts().to_dict()}")
    
    print("\nPhase 5: Saving results...")
    
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        
        # Save comprehensive results
        sensitivity_file = os.path.join(
            save_path, 
            f'{target_name.lower().replace(" ", "_")}_sensitivity_{population_scenario}.xlsx'
        )
        
        with pd.ExcelWriter(sensitivity_file, engine='openpyxl') as writer:
            # Sheet 1: All configurations summary
            results_df.to_excel(writer, sheet_name='All_Configurations', index=False)
            
            # Sheet 2: Simplified errors - Mean CV MAE and Train-Test MAE per config
            errors_df.to_excel(writer, sheet_name='Model_Errors', index=False)
            
            # Sheet 3: Recommendations
            rec_frames = list(recommendations_per_horizon.values())
            if rec_frames:
                all_recs = pd.concat(rec_frames, ignore_index=True)
                all_recs.to_excel(writer, sheet_name='Recommendations', index=False)
            
            # Sheet 4: Configuration summary statistics
            config_summary = results_df.groupby('config_id').agg({
                'cv_traintest_agree': 'mean',
                'cv_consistency': 'mean'
            }).reset_index()
            config_summary.columns = ['config_id', 'avg_cv_tt_agreement', 'avg_cv_consistency']
            
            # Add parameter details
            param_details = results_df.groupby('config_id')[param_names].first().reset_index()
            config_summary = config_summary.merge(param_details, on='config_id')
            config_summary.to_excel(writer, sheet_name='Config_Summary', index=False)
        
        print(f"✅ Sensitivity analysis saved: {sensitivity_file}")
        
        # Save production forecasts
        for horizon, forecasts in production_forecasts.items():
            horizon_file = os.path.join(
                save_path,
                f'{target_name.lower().replace(" ", "_")}_production_h{horizon}_{population_scenario}.xlsx'
            )
            
            with pd.ExcelWriter(horizon_file, engine='openpyxl') as writer:
                forecasts['all_models'].reset_index().to_excel(
                    writer, sheet_name='All_Models', index=False
                )
                forecasts['best_model'].to_excel(
                    writer, sheet_name='Best_Model_Forecast', index=False
                )
                forecasts['recommendations'].to_excel(
                    writer, sheet_name='Model_Selection', index=False
                )
            
            print(f"✅ Production forecasts (h={horizon}) saved: {horizon_file}")
    else:
        print("   ⚠️ No save_path provided - skipping file export")
    
    print("\n" + "=" * 80)
    print("✅ SENSITIVITY ANALYSIS COMPLETE!")
    print("=" * 80)
    
    return {
        'sensitivity_results': results_df,
        'model_errors': errors_df,
        'production_forecasts': production_forecasts,
        'recommendations_per_horizon': recommendations_per_horizon
    }

In [56]:
def plot_top_atc2_forecasts(
    df_historical: pd.DataFrame,
    df_forecast: pd.DataFrame,
    state: str,
    target_col: str = 'Units Reimbursed',
    model_cols: List[str] = ['SARIMAX', 'Naive'],
    top_n: int = 12,
    figsize: Tuple[int, int] = (18, 14),
    show_ci: bool = True,
    save_path: Optional[str] = None
) -> plt.Figure:
    """Plot top N ATC-2 classes with multiple model comparison (max 4)."""
    if len(model_cols) > 4:
        model_cols = model_cols[:4]
    
    df_hist = df_historical.copy()
    df_fore = df_forecast.copy()
    if 'ds' not in df_fore.columns and 'ds' in df_fore.index.names:
        df_fore = df_fore.reset_index()
    df_hist['ds'] = pd.to_datetime(df_hist['ds'])
    df_fore['ds'] = pd.to_datetime(df_fore['ds'])
    df_hist = df_hist[df_hist['unique_id'].str.startswith(f'{state}_')]
    df_fore = df_fore[df_fore['unique_id'].str.startswith(f'{state}_')]
    
    top_classes = df_hist.groupby('unique_id')['y'].sum().sort_values(ascending=False).head(top_n).index.tolist()
    n_cols, n_rows = 4, (top_n + 3) // 4
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize, squeeze=False)
    axes = axes.flatten()
    
    for idx, uid in enumerate(top_classes):
        ax = axes[idx]
        atc_class = uid.split('_')[1]
        hist_data = df_hist[df_hist['unique_id'] == uid].sort_values('ds')
        fore_data = df_fore[df_fore['unique_id'] == uid].sort_values('ds')
        scale = 1e6 if hist_data['y'].max() > 1e6 else 1e3 if hist_data['y'].max() > 1e3 else 1
        scale_label = 'M' if scale == 1e6 else 'K' if scale == 1e3 else ''
        
        ax.plot(hist_data['ds'], hist_data['y']/scale, color=HISTORICAL_COLOR, lw=1.5, marker='o', ms=2, label='Historical')
        last_hist = hist_data.iloc[-1] if len(hist_data) > 0 else None
        
        for mi, mc in enumerate(model_cols):
            if mc not in fore_data.columns or len(fore_data) == 0: continue
            color = MODEL_COLORS[mi]
            if last_hist is not None:
                ax.plot([last_hist['ds'], fore_data['ds'].iloc[0]], [last_hist['y']/scale, fore_data[mc].iloc[0]/scale],
                       color=color, lw=1.5, ls='--', alpha=0.7)
            ax.plot(fore_data['ds'], fore_data[mc]/scale, color=color, lw=1.5, marker='s', ms=3, ls='--', label=mc)
            if show_ci and mi == 0:
                lo, hi = f'{mc}-lo-95', f'{mc}-hi-95'
                if lo in fore_data.columns and hi in fore_data.columns:
                    ax.fill_between(fore_data['ds'], fore_data[lo]/scale, fore_data[hi]/scale, color=color, alpha=0.1)
        
        if last_hist is not None:
            ax.axvline(x=last_hist['ds'], color='gray', ls=':', alpha=0.5)
        ax.set_title(f'{atc_class}', fontsize=10, fontweight='bold')
        ax.set_ylabel(scale_label)
        ax.tick_params(axis='x', rotation=45, labelsize=7)
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(loc='upper left', fontsize=7)
    
    for idx in range(len(top_classes), len(axes)):
        axes[idx].set_visible(False)
    
    fig.suptitle(f'{state} - Top {top_n} ATC-2: {target_col}\nModels: {", ".join(model_cols)}', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✔ Saved: {save_path}")
    plt.show()
    return fig

def plot_single_atc2_forecast(
    df_historical: pd.DataFrame,
    df_forecast: pd.DataFrame,
    state: str,
    atc_class: str,
    target_col: str = 'Units Reimbursed',
    model_cols: List[str] = ['SARIMAX', 'Naive'],
    show_ci: bool = True,
    figsize: Tuple[int, int] = (14, 7),
    save_path: Optional[str] = None
) -> plt.Figure:
    """Detailed plot for single ATC-2 class with multiple models."""
    if len(model_cols) > 4:
        model_cols = model_cols[:4]
    
    df_hist = df_historical.copy()
    df_fore = df_forecast.copy()
    if 'ds' not in df_fore.columns and 'ds' in df_fore.index.names:
        df_fore = df_fore.reset_index()
    df_hist['ds'] = pd.to_datetime(df_hist['ds'])
    df_fore['ds'] = pd.to_datetime(df_fore['ds'])
    
    uid = f'{state}_{atc_class}'
    hist_data = df_hist[df_hist['unique_id'] == uid].sort_values('ds')
    fore_data = df_fore[df_fore['unique_id'] == uid].sort_values('ds')
    
    if len(hist_data) == 0:
        print(f"No data for {uid}")
        return None
    
    fig, ax = plt.subplots(figsize=figsize)
    scale = 1e6 if hist_data['y'].max() > 1e6 else 1e3 if hist_data['y'].max() > 1e3 else 1
    scale_label = 'Millions' if scale == 1e6 else 'Thousands' if scale == 1e3 else ''
    
    ax.plot(hist_data['ds'], hist_data['y']/scale, color=HISTORICAL_COLOR, lw=2.5, marker='o', ms=5, label='Historical', zorder=10)
    last_hist = hist_data.iloc[-1]
    
    for mi, mc in enumerate(model_cols):
        if mc not in fore_data.columns or len(fore_data) == 0:
            print(f"Warning: '{mc}' not found.")
            continue
        color = MODEL_COLORS[mi]
        ax.plot([last_hist['ds'], fore_data['ds'].iloc[0]], [last_hist['y']/scale, fore_data[mc].iloc[0]/scale],
               color=color, lw=2, ls='--', alpha=0.7)
        ax.plot(fore_data['ds'], fore_data[mc]/scale, color=color, lw=2, marker='s', ms=6, ls='--', label=mc)
        lo, hi = f'{mc}-lo-95', f'{mc}-hi-95'
        if show_ci and lo in fore_data.columns and hi in fore_data.columns:
            ax.fill_between(fore_data['ds'], fore_data[lo]/scale, fore_data[hi]/scale, color=color, alpha=0.15, label=f'{mc} 95% CI')
    
    ax.axvline(x=last_hist['ds'], color='gray', ls=':', alpha=0.7, lw=2, label='Forecast Start')
    ax.set_xlabel('Year-Quarter', fontsize=12)
    ax.set_ylabel(f'{target_col} ({scale_label})', fontsize=12)
    ax.set_title(f'{state} - {atc_class}: {target_col}\nModels: {", ".join(model_cols)}', fontsize=14, fontweight='bold')
    ax.legend(loc='upper left', fontsize=10, ncol=2)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✔ Saved: {save_path}")
    plt.show()
    return fig

In [57]:
filepath_withpop = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\P1_withpop.csv"
save_path = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\TMF\\"

In [58]:
'''
# Configure
config = ForecastConfig(
    h=8,
    season_length=4,
    n_windows=3,
    train_size=None,
    test_size=8,  #Setting it to the same h
    n_samples=4,
    confidence_level=95,
    models_to_plot=['Naive', 'SARIMAX']
)

# Run forecasting for Units Reimbursed with all three population scenarios
results_units = forecast_with_population_scenarios(
    filepath=filepath_withpop,
    target_col='Units Reimbursed',
    states=['IN'],  # Or None for all states
    config=config,
    horizons=[8],
    save_path=save_path
)

# Run forecasting for Number of Prescriptions
results_prescriptions = forecast_with_population_scenarios(
    filepath=filepath_withpop,
    target_col='Number of Prescriptions',
    states=['IL'],
    config=config,
    horizons=[8],
    save_path=save_path
)
'''

"\n# Configure\nconfig = ForecastConfig(\n    h=8,\n    season_length=4,\n    n_windows=3,\n    train_size=None,\n    test_size=8,  #Setting it to the same h\n    n_samples=4,\n    confidence_level=95,\n    models_to_plot=['Naive', 'SARIMAX']\n)\n\n# Run forecasting for Units Reimbursed with all three population scenarios\nresults_units = forecast_with_population_scenarios(\n    filepath=filepath_withpop,\n    target_col='Units Reimbursed',\n    states=['IN'],  # Or None for all states\n    config=config,\n    horizons=[8],\n    save_path=save_path\n)\n\n# Run forecasting for Number of Prescriptions\nresults_prescriptions = forecast_with_population_scenarios(\n    filepath=filepath_withpop,\n    target_col='Number of Prescriptions',\n    states=['IL'],\n    config=config,\n    horizons=[8],\n    save_path=save_path\n)\n"

In [59]:
# Example usage
results = sensitivity_analysis_with_exog(
    filepath=filepath_withpop,
    target_col='Number of Prescriptions',
    states=['MI'],
    param_grid={
        'h': [4, 8],
        'train_size': [None], #Training all the dataset for Cross-Validation, otherwise the n_windows will be too small
        'n_windows': [2],
        'step_size': [4],
        'test_size': [4, 8],  #Setting it to the same as h
    },
    production_horizons=[4, 8],
    population_scenario='point',  # 'point' or 'low_95' or 'high_95'
    save_path=save_path
)

SENSITIVITY ANALYSIS WITH EXOGENOUS VARIABLES: Number of Prescriptions
Population Scenario: POINT
Testing 4 configurations

Phase 1: Loading and preparing data...
Filtering data for states: ['MI']
Filtered to 2663 rows across ['MI'] state(s)

Unique series: 84

Date range: 2017-01-01 00:00:00 to 2024-10-01 00:00:00

Total observations: 2663
   point: 672 future population records
   low_95: 672 future population records
   high_95: 672 future population records

Population scenario loaded: point

Population records: 3335

Phase 2: Running sensitivity analysis...

────────────────────────────────────────────────────────────────────────────────
Configuration 1/4: {'h': 4, 'train_size': None, 'n_windows': 2, 'step_size': 4, 'test_size': 4}
────────────────────────────────────────────────────────────────────────────────

Running train-test evaluation...

📈 Best Models (Train-Test Split - Number of Prescriptions (with population)):
best_model
Naive              29
HistoricAverage    20
SARI

### Visualization and Plots

In [ ]:
# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Color palette for statistical models
STATS_MODEL_COLORS = {
    'SARIMAX': '#2E86AB',
    'ARIMAX': '#E94F37',
    'AutoARIMA': '#28A745',
    'SeasonalNaive': '#9B59B6',
    'Naive': '#F39C12',
    'HistoricAverage': '#1ABC9C',
    'WindowAverage': '#E74C3C',
    'DOT': '#3498DB',
    'HoltWinters': '#8E44AD',
}

# Extract results from sensitivity analysis
sensitivity_results = results['sensitivity_results']
errors_df = results['model_errors']
recommendations_per_horizon = results['recommendations_per_horizon']

# IMPORTANT: Extract model names from column names
# Columns are like: SARIMAX_cv_mean_mae, Naive_tt_mae, etc.
cv_mae_cols = [col for col in errors_df.columns if col.endswith('_cv_mean_mae')]
tt_mae_cols = [col for col in errors_df.columns if col.endswith('_tt_mae')]

# Extract model names
stats_model_names = [col.replace('_cv_mean_mae', '') for col in cv_mae_cols]
print(f"\n✓ Detected models: {stats_model_names}")

# Get recommendations for the primary horizon
primary_horizon = 8
if primary_horizon in recommendations_per_horizon:
    stats_recommendations_df = recommendations_per_horizon[primary_horizon]
else:
    primary_horizon = list(recommendations_per_horizon.keys())[0]
    stats_recommendations_df = recommendations_per_horizon[primary_horizon]

print(f"✓ Loaded recommendations for horizon={primary_horizon}")
print(f"✓ Total series: {len(stats_recommendations_df)}")
print(f"✓ errors_df shape: {errors_df.shape}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Calculate mean CV MAE for each model across all series
cv_means = {}
cv_stds = {}
tt_means = {}
tt_stds = {}

for model in stats_model_names:
    cv_col = f'{model}_cv_mean_mae'
    tt_col = f'{model}_tt_mae'
    
    if cv_col in errors_df.columns:
        cv_means[model] = errors_df[cv_col].mean()
        cv_stds[model] = errors_df[cv_col].std()
    
    if tt_col in errors_df.columns:
        tt_means[model] = errors_df[tt_col].mean()
        tt_stds[model] = errors_df[tt_col].std()

# Sort by CV MAE
cv_sorted = dict(sorted(cv_means.items(), key=lambda x: x[1]))
tt_sorted = dict(sorted(tt_means.items(), key=lambda x: x[1]))

# CV MAE Chart
ax1 = axes[0]
models_cv = list(cv_sorted.keys())
values_cv = list(cv_sorted.values())
errors_cv = [cv_stds[m] for m in models_cv]
colors_cv = [STATS_MODEL_COLORS.get(m, '#666666') for m in models_cv]

bars1 = ax1.barh(models_cv, values_cv, xerr=errors_cv, color=colors_cv, 
                  capsize=5, alpha=0.8, edgecolor='white', linewidth=1.5)
ax1.set_xlabel('Mean CV MAE')
ax1.set_title('Average CV MAE by Model\n(Lower is Better)', fontweight='bold')
ax1.axvline(x=min(values_cv), color='green', linestyle='--', alpha=0.5, 
            label=f'Best: {min(values_cv):,.0f}')

for bar, val in zip(bars1, values_cv):
    ax1.text(val + max(errors_cv)*0.1, bar.get_y() + bar.get_height()/2,
             f'{val:,.0f}', va='center', fontsize=10)
ax1.legend()

# Train-Test MAE Chart
ax2 = axes[1]
models_tt = list(tt_sorted.keys())
values_tt = list(tt_sorted.values())
errors_tt = [tt_stds[m] for m in models_tt]
colors_tt = [STATS_MODEL_COLORS.get(m, '#666666') for m in models_tt]

bars2 = ax2.barh(models_tt, values_tt, xerr=errors_tt, color=colors_tt,
                  capsize=5, alpha=0.8, edgecolor='white', linewidth=1.5)
ax2.set_xlabel('Mean Train-Test MAE')
ax2.set_title('Average Train-Test MAE by Model\n(Lower is Better)', fontweight='bold')
ax2.axvline(x=min(values_tt), color='green', linestyle='--', alpha=0.5,
            label=f'Best: {min(values_tt):,.0f}')

for bar, val in zip(bars2, values_tt):
    ax2.text(val + max(errors_tt)*0.1, bar.get_y() + bar.get_height()/2,
             f'{val:,.0f}', va='center', fontsize=10)
ax2.legend()

plt.tight_layout()
plt.savefig(os.path.join(save_path, 'stats_model_performance_overview.png'), dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: stats_model_performance_overview.png")

In [ ]:
print("\n" + "=" * 80)
print("STATISTICAL MODEL PERFORMANCE STATISTICS")
print("=" * 80)

# Build statistics table
stats_data = []
for model in stats_model_names:
    cv_col = f'{model}_cv_mean_mae'
    tt_col = f'{model}_tt_mae'
    
    row = {'Model': model}
    
    if cv_col in errors_df.columns:
        cv_vals = errors_df[cv_col].dropna()
        row['CV_MAE_Mean'] = cv_vals.mean()
        row['CV_MAE_Median'] = cv_vals.median()
        row['CV_MAE_Std'] = cv_vals.std()
        row['CV_MAE_Min'] = cv_vals.min()
        row['CV_MAE_Max'] = cv_vals.max()
    
    if tt_col in errors_df.columns:
        tt_vals = errors_df[tt_col].dropna()
        row['TT_MAE_Mean'] = tt_vals.mean()
        row['TT_MAE_Median'] = tt_vals.median()
        row['TT_MAE_Std'] = tt_vals.std()
        row['TT_MAE_Min'] = tt_vals.min()
        row['TT_MAE_Max'] = tt_vals.max()
    
    stats_data.append(row)

stats_summary = pd.DataFrame(stats_data)

print("\n📊 Cross-Validation MAE Statistics:")
print("-" * 90)
print(f"{'Model':<20} {'Mean':>12} {'Median':>12} {'Std':>12} {'Min':>12} {'Max':>12}")
print("-" * 90)
for _, row in stats_summary.sort_values('CV_MAE_Mean').iterrows():
    print(f"{row['Model']:<20} {row['CV_MAE_Mean']:>12,.0f} {row['CV_MAE_Median']:>12,.0f} "
          f"{row['CV_MAE_Std']:>12,.0f} {row['CV_MAE_Min']:>12,.0f} {row['CV_MAE_Max']:>12,.0f}")

print("\n📊 Train-Test MAE Statistics:")
print("-" * 90)
print(f"{'Model':<20} {'Mean':>12} {'Median':>12} {'Std':>12} {'Min':>12} {'Max':>12}")
print("-" * 90)
for _, row in stats_summary.sort_values('TT_MAE_Mean').iterrows():
    print(f"{row['Model']:<20} {row['TT_MAE_Mean']:>12,.0f} {row['TT_MAE_Median']:>12,.0f} "
          f"{row['TT_MAE_Std']:>12,.0f} {row['TT_MAE_Min']:>12,.0f} {row['TT_MAE_Max']:>12,.0f}")

stats_summary.to_csv(os.path.join(save_path, 'stats_model_statistics.csv'), index=False)
print("\n✓ Saved: stats_model_statistics.csv")

In [ ]:
# Reshape data for box plots (wide to long format)
cv_data_long = []
tt_data_long = []

for model in stats_model_names:
    cv_col = f'{model}_cv_mean_mae'
    tt_col = f'{model}_tt_mae'
    
    if cv_col in errors_df.columns:
        for val in errors_df[cv_col].dropna():
            cv_data_long.append({'Model': model, 'CV_MAE': val})
    
    if tt_col in errors_df.columns:
        for val in errors_df[tt_col].dropna():
            tt_data_long.append({'Model': model, 'TT_MAE': val})

cv_long_df = pd.DataFrame(cv_data_long)
tt_long_df = pd.DataFrame(tt_data_long)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Sort by median
model_order_cv = cv_long_df.groupby('Model')['CV_MAE'].median().sort_values().index.tolist()
model_order_tt = tt_long_df.groupby('Model')['TT_MAE'].median().sort_values().index.tolist()

# CV MAE Box Plot
ax1 = axes[0]
palette_cv = [STATS_MODEL_COLORS.get(m, '#666666') for m in model_order_cv]
sns.boxplot(data=cv_long_df, x='Model', y='CV_MAE', order=model_order_cv,
            palette=palette_cv, ax=ax1, showfliers=True, flierprops={'alpha': 0.5})
ax1.set_title('CV MAE Distribution by Model\n(Sorted by Median)', fontweight='bold')
ax1.set_ylabel('CV MAE')
ax1.tick_params(axis='x', rotation=30)

# Add median labels
medians_cv = cv_long_df.groupby('Model')['CV_MAE'].median()[model_order_cv]
for i, (model, median) in enumerate(medians_cv.items()):
    ax1.annotate(f'{median:,.0f}', xy=(i, median), xytext=(0, 5),
                 textcoords='offset points', ha='center', fontsize=9, fontweight='bold')

# Train-Test MAE Box Plot
ax2 = axes[1]
palette_tt = [STATS_MODEL_COLORS.get(m, '#666666') for m in model_order_tt]
sns.boxplot(data=tt_long_df, x='Model', y='TT_MAE', order=model_order_tt,
            palette=palette_tt, ax=ax2, showfliers=True, flierprops={'alpha': 0.5})
ax2.set_title('Train-Test MAE Distribution by Model\n(Sorted by Median)', fontweight='bold')
ax2.set_ylabel('Train-Test MAE')
ax2.tick_params(axis='x', rotation=30)

medians_tt = tt_long_df.groupby('Model')['TT_MAE'].median()[model_order_tt]
for i, (model, median) in enumerate(medians_tt.items()):
    ax2.annotate(f'{median:,.0f}', xy=(i, median), xytext=(0, 5),
                 textcoords='offset points', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(save_path, 'stats_error_distribution_boxplot.png'), dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: stats_error_distribution_boxplot.png")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

best_model_counts = stats_recommendations_df['recommended_model'].value_counts()
colors_pie = [STATS_MODEL_COLORS.get(m, '#666666') for m in best_model_counts.index]

# Pie chart
ax1 = axes[0]
wedges, texts, autotexts = ax1.pie(
    best_model_counts.values,
    labels=best_model_counts.index,
    autopct='%1.1f%%',
    colors=colors_pie,
    explode=[0.02] * len(best_model_counts),
    startangle=90,
    textprops={'fontsize': 10}
)
ax1.set_title(f'Best Model Distribution (h={primary_horizon})\n{len(stats_recommendations_df)} Series', fontweight='bold')

# Bar chart
ax2 = axes[1]
bars = ax2.bar(best_model_counts.index, best_model_counts.values, color=colors_pie,
               edgecolor='white', linewidth=1.5)
ax2.set_ylabel('Number of Series')
ax2.set_xlabel('Model')
ax2.set_title('Series Count per Best Model', fontweight='bold')
ax2.tick_params(axis='x', rotation=30)

for bar, count in zip(bars, best_model_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(save_path, 'stats_best_model_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Best Model Distribution (Statistical):")
print("-" * 50)
for model, count in best_model_counts.items():
    pct = count / len(stats_recommendations_df) * 100
    print(f"   {model:<20}: {count:>3} series ({pct:>5.1f}%)")
print("\n✓ Saved: stats_best_model_distribution.png")

In [ ]:
if 'confidence' in stats_recommendations_df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    confidence_counts = stats_recommendations_df['confidence'].value_counts()
    confidence_order = ['High', 'Medium', 'Low']
    confidence_colors = {'High': '#28A745', 'Medium': '#F39C12', 'Low': '#E94F37'}
    
    confidence_counts = confidence_counts.reindex(confidence_order).dropna()
    colors_conf = [confidence_colors.get(c, '#666666') for c in confidence_counts.index]
    
    bars = ax.bar(confidence_counts.index, confidence_counts.values, color=colors_conf,
                  edgecolor='white', linewidth=2)
    ax.set_ylabel('Number of Series')
    ax.set_xlabel('Confidence Level')
    ax.set_title('Recommendation Confidence Distribution\n(Based on MAE margin between best and 2nd best)', 
                 fontweight='bold')
    
    for bar, count in zip(bars, confidence_counts.values):
        pct = count / len(stats_recommendations_df) * 100
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{count}\n({pct:.1f}%)', ha='center', va='bottom', fontsize=11)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, 'stats_confidence_distribution.png'), dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Confidence Distribution:")
    for conf in ['High', 'Medium', 'Low']:
        count = (stats_recommendations_df['confidence'] == conf).sum()
        if count > 0:
            pct = count / len(stats_recommendations_df) * 100
            print(f"   {conf}: {count} series ({pct:.1f}%)")
    print("\n✓ Saved: stats_confidence_distribution.png")
else:
    print("Note: 'confidence' column not available in recommendations")

In [ ]:
if 'cv_traintest_agree' in sensitivity_results.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Agreement by model (which CV best model)
    ax1 = axes[0]
    agreement_by_model = sensitivity_results.groupby('cv_best_model')['cv_traintest_agree'].mean() * 100
    agreement_by_model = agreement_by_model.sort_values(ascending=False)
    
    colors = [STATS_MODEL_COLORS.get(m, '#666666') for m in agreement_by_model.index]
    bars = ax1.bar(agreement_by_model.index, agreement_by_model.values, color=colors,
                   edgecolor='white', linewidth=1.5)
    ax1.set_ylabel('Agreement Rate (%)')
    ax1.set_xlabel('Model (CV Best)')
    ax1.set_title('CV vs Train-Test Agreement by Model\n(Higher = More Consistent)', fontweight='bold')
    ax1.tick_params(axis='x', rotation=30)
    ax1.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% baseline')
    ax1.legend()
    
    for bar, val in zip(bars, agreement_by_model.values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val:.1f}%', ha='center', va='bottom', fontsize=10)
    
    # Overall agreement pie
    ax2 = axes[1]
    agree_count = sensitivity_results['cv_traintest_agree'].sum()
    disagree_count = len(sensitivity_results) - agree_count
    
    ax2.pie([agree_count, disagree_count], 
            labels=['Agree', 'Disagree'],
            autopct='%1.1f%%',
            colors=['#28A745', '#E94F37'],
            explode=[0.02, 0.02],
            startangle=90,
            textprops={'fontsize': 12})
    ax2.set_title(f'Overall CV vs Train-Test Agreement\n({agree_count}/{len(sensitivity_results)} series)', 
                  fontweight='bold')
    
    plt.tight_layout()
    #plt.savefig(os.path.join(save_path, 'stats_cv_traintest_agreement.png'), dpi=150, bbox_inches='tight')
    plt.show()
    
    overall_agreement = sensitivity_results['cv_traintest_agree'].mean() * 100
    print(f"\n📊 Overall CV vs Train-Test Agreement: {overall_agreement:.1f}%")
    print("\n✓ Saved: stats_cv_traintest_agreement.png")

In [ ]:
# Calculate which model ranks best for each series
rankings_data = []

for idx, row in errors_df.iterrows():
    uid = row['unique_id']
    
    # Get CV MAE values for each model
    cv_maes = {}
    for model in stats_model_names:
        cv_col = f'{model}_cv_mean_mae'
        if cv_col in row.index and pd.notna(row[cv_col]):
            cv_maes[model] = row[cv_col]
    
    # Rank models (1 = best)
    if cv_maes:
        sorted_models = sorted(cv_maes.items(), key=lambda x: x[1])
        for rank, (model, mae) in enumerate(sorted_models, 1):
            rankings_data.append({'unique_id': uid, 'Model': model, 'Rank': rank})

rankings_df = pd.DataFrame(rankings_data)

# Count rankings
rank_counts = pd.DataFrame(index=stats_model_names, columns=['1st', '2nd', '3rd', '4th', '5th', '6th'])
for model in stats_model_names:
    model_ranks = rankings_df[rankings_df['Model'] == model]['Rank']
    for rank_pos, rank_label in enumerate(['1st', '2nd', '3rd', '4th', '5th', '6th'], 1):
        rank_counts.loc[model, rank_label] = (model_ranks == rank_pos).sum()

# Keep only columns with data
rank_counts = rank_counts.dropna(axis=1, how='all').fillna(0).astype(int)

fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(rank_counts, annot=True, fmt='d', cmap='RdYlGn_r', ax=ax,
            cbar_kws={'label': 'Number of Series'}, linewidths=0.5,
            annot_kws={'size': 12, 'weight': 'bold'})
ax.set_title('Model Rankings Across Series (CV MAE)\n(How often does each model rank 1st, 2nd, etc.)', 
             fontweight='bold', pad=20)
ax.set_xlabel('Rank Position')
ax.set_ylabel('Model')

plt.tight_layout()
plt.savefig(os.path.join(save_path, 'stats_model_rankings_heatmap.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Model Ranking Summary:")
print("-" * 60)
print(rank_counts.to_string())
print("\n✓ Saved: stats_model_rankings_heatmap.png")

In [ ]:
if 'avg_mae' in stats_recommendations_df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    top_n = 10
    sorted_by_mae = stats_recommendations_df.sort_values('avg_mae')
    
    # Best performers
    ax1 = axes[0]
    top_performers = sorted_by_mae.head(top_n)
    colors_best = [STATS_MODEL_COLORS.get(m, '#666666') for m in top_performers['recommended_model']]
    
    bars1 = ax1.barh(range(len(top_performers)), top_performers['avg_mae'].values, 
                     color=colors_best, alpha=0.8)
    ax1.set_yticks(range(len(top_performers)))
    labels = [f"{row['unique_id'].split('_')[-1][:12]} ({row['recommended_model']})" 
              for _, row in top_performers.iterrows()]
    ax1.set_yticklabels(labels, fontsize=9)
    ax1.set_xlabel('MAE')
    ax1.set_title(f'Top {top_n} Best Performing Series\n(Lowest MAE)', fontweight='bold')
    ax1.invert_yaxis()
    
    # Worst performers
    ax2 = axes[1]
    bottom_performers = sorted_by_mae.tail(top_n).sort_values('avg_mae', ascending=False)
    colors_worst = [STATS_MODEL_COLORS.get(m, '#666666') for m in bottom_performers['recommended_model']]
    
    bars2 = ax2.barh(range(len(bottom_performers)), bottom_performers['avg_mae'].values,
                     color=colors_worst, alpha=0.8)
    ax2.set_yticks(range(len(bottom_performers)))
    labels = [f"{row['unique_id'].split('_')[-1][:12]} ({row['recommended_model']})"
              for _, row in bottom_performers.iterrows()]
    ax2.set_yticklabels(labels, fontsize=9)
    ax2.set_xlabel('MAE')
    ax2.set_title(f'Top {top_n} Worst Performing Series\n(Highest MAE)', fontweight='bold')
    ax2.invert_yaxis()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, 'stats_best_worst_series.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: stats_best_worst_series.png")


# -----------------------------------------------------------------------------
# CELL 6.10: Final Summary
# -----------------------------------------------------------------------------

print("\n" + "=" * 80)
print("📈 STATISTICAL FORECASTING MODEL EVALUATION SUMMARY")
print("=" * 80)

# Best overall model by CV MAE
best_cv_model = min(cv_means, key=cv_means.get)
print(f"\n🏆 Best Overall Model (by Mean CV MAE): {best_cv_model}")
print(f"   Mean CV MAE: {cv_means[best_cv_model]:,.0f}")

# Most frequently best
top_model = stats_recommendations_df['recommended_model'].value_counts().idxmax()
top_count = stats_recommendations_df['recommended_model'].value_counts().max()
print(f"\n📊 Most Frequently Best Model: {top_model}")
print(f"   Selected for: {top_count}/{len(stats_recommendations_df)} series ({top_count/len(stats_recommendations_df)*100:.1f}%)")

# Distribution
print(f"\n📊 Best Model per Series (h={primary_horizon}):")
for model, count in stats_recommendations_df['recommended_model'].value_counts().items():
    pct = count / len(stats_recommendations_df) * 100
    print(f"   {model}: {count} series ({pct:.1f}%)")

# Confidence
if 'confidence' in stats_recommendations_df.columns:
    print(f"\n📊 Recommendation Confidence:")
    for conf in ['High', 'Medium', 'Low']:
        count = (stats_recommendations_df['confidence'] == conf).sum()
        if count > 0:
            pct = count / len(stats_recommendations_df) * 100
            print(f"   {conf}: {count} series ({pct:.1f}%)")

# Agreement
if 'cv_traintest_agree' in sensitivity_results.columns:
    agree_pct = sensitivity_results['cv_traintest_agree'].mean() * 100
    print(f"\n📊 CV vs Train-Test Agreement: {agree_pct:.1f}%")

print("\n" + "=" * 80)
print("✅ Section 6 Complete - All visualizations saved!")
print("=" * 80)